langsmith digunakan untuk tracking block chain yang digunakan pada langchain

In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [6]:
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")

1. Model

In [8]:
from langchain_community.chat_models import ChatOllama

/Users/a/Programming/Langchain-Project/my-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
llm_model = ChatOllama(
    model='llama3:8b',
    temperature=0
)

/var/folders/05/16tm42991snf6c27rs7nssdh0000gn/T/ipykernel_24845/2586603255.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm_model = ChatOllama(


2. prompt template

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [11]:
prompt = ChatPromptTemplate.from_template(
    """Kamu adalah asisten chatbot yang membantu. 
    Tugasmu adalah menjawab pertanyaan berikut secara langsung dalam Bahasa Indonesia. 
    Meskipun pertanyaan diberikan dalam Bahasa Inggris atau bahasa lain, kamu WAJIB menjawabnya dalam Bahasa Indonesia.
    
    Pertanyaan: {input}
    Jawaban:"""
)

In [13]:
interface_user = "apa itu ninjutsu"

In [14]:
chain = prompt | llm_model | StrOutputParser()

In [15]:
respone = chain.invoke({"input": interface_user})
print(respone)

Ninjutsu adalah seni bela diri tradisional Jepang yang dikembangkan oleh ninja. Istilah "ninjutsu" sendiri berarti "ilmu ninja". Ninjutsu meliputi berbagai teknik, seperti gerakan tubuh, strategi, dan psikologi, yang digunakan untuk mencapai tujuan tertentu, seperti pengintaian, perang, atau penyelamatan.


3. Embedding model -> digunakan untuk mengubah external data ke dalam bentuk vector 

In [17]:
# memanggil Indobert dari transformer
from transformers import BertTokenizer, AutoModel

tokenizer = BertTokenizer.from_pretrained("Indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [18]:
# membuat class model embdding
from typing import List 
from langchain_core.embeddings import Embeddings
import torch

class IndoBertEmbeddings(Embeddings):
    def __init__(self, model_name="indobenchmark/indobert-base-p1"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()


    def _generate_embedding(self, text: str) -> List[float]:
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        with torch.no_grad():
            outputs = self.model(**inputs)

        # polling token menjadi satu vector kalimat
        token_embeddings = outputs.last_hidden_state

        # melakukan mean polling
        sentence_embeddings = token_embeddings.mean(dim=1)

        # konversi ke list python
        return sentence_embeddings.squeeze().tolist()
    

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._generate_embedding(text) for text in texts]
    

    # metode untuk pencarian query pada chroma 
    def embed_query(self, text: str) -> List[float]:
        return self._generate_embedding(text)


In [20]:
embeddings = IndoBertEmbeddings()

4. Vector store (kita akan menggunakan chromadb yang sebelumnya telah disetup)

In [21]:
from langchain_chroma import Chroma

In [23]:
vectore_store = Chroma(
    collection_name="chroma-with-langchain",
    embedding_function=embeddings,
    persist_directory="../Langchain-Project/notebooks/chroma_langchain_db"
)

5. Indexing (Load -> split -> store)

![image.png](https://mintcdn.com/langchain-5e9cc07a/I6RpA28iE233vhYX/images/rag_indexing.png?w=840&fit=max&auto=format&n=I6RpA28iE233vhYX&q=85&s=1838328a870c7353c42bf1cc2290a779)

In [25]:
# load documents dan kita ubah kedalam bentuk document standart langchain yaitu document object 
import bs4
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post_header", "post-content"))
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),  # data diambil dari blognya lilianweng
    bs_kwargs={"parse_only": bs4_strainer}
)


In [27]:
docs = loader.load()

In [28]:
assert len(docs) == 1
print(f"Total character: {len(docs[0].page_content)}")

Total character: 42964


In [31]:
print(docs[0].page_content[:600])


      LLM Powered Autonomous Agents
    Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large t
